In [1]:
import pandas as pd
import pandapower as pp
import numpy as np
import copy
import time

In [10]:
# ------------------------------
# 1. Load the Network
# ------------------------------
net = pp.from_json("../../data/task1_security_classification/digital_twin_ext_grid.json")

In [24]:
net

This pandapower network includes the following parameter tables:
   - bus (35 elements)
   - load (20 elements)
   - sgen (110 elements)
   - gen (135 elements)
   - ext_grid (1 element)
   - line (46 elements)

In [20]:
# ------------------------------
# 2. Load Time-Series Data
# ------------------------------
distributed_loads_df = pd.read_csv("../../data/task1_security_classification/distributed_loads_uniform.csv")
distributed_generation_df = pd.read_csv("../../data/task1_security_classification/distributed_generators.csv")

# Convert timestamps
distributed_loads_df["timestamp"] = pd.to_datetime(distributed_loads_df["timestamp"])
distributed_generation_df["timestamp"] = pd.to_datetime(distributed_generation_df["timestamp"])

distributed_loads_df.set_index("timestamp", inplace=True)
distributed_generation_df.set_index("timestamp", inplace=True)

common_timestamps = distributed_loads_df.index.intersection(distributed_generation_df.index)
distributed_loads_df = distributed_loads_df.loc[common_timestamps]
distributed_generation_df = distributed_generation_df.loc[common_timestamps]

In [21]:
# ------------------------------
# 3. Prepare Featues Storage
# ------------------------------
features = {
    "timestamp": [],
    "status": [],  # secure or insecure (label)

    # Base-case results
    "max_line_loading_percent_basecase": [],
    "min_bus_voltage_pu_basecase": [],
    "max_bus_voltage_pu_basecase": [],
    
    # Contingency (N-1) worst cases
    "max_line_loading_percent_contingency": [],
    "min_bus_voltage_pu_contingency": [],
    "max_bus_voltage_pu_contingency": []
}

# Per-load p_mw columns
for load_idx in net.load.index:
    features[f"load_{load_idx}_p_mw"] = []

# Per-gen p_mw columns
for gen_idx in net.gen.index:
    features[f"gen_{gen_idx}_p_mw"] = []

# Per-sgen p_mw  columns
for sgen_idx in net.sgen.index:
    features[f"sgen_{sgen_idx}_p_mw"] = []

In [ ]:
# ------------------------------
# 4. Time-Series + N-1 Loop
# ------------------------------
start_time = time.time()
# Decide which elements to include in N-1:
# Lines, Transformers, and Generators (gen). 
contingency_lines = [idx for idx in net.line.index if idx != 45]  # Exclude Slack-to-Bus line
contingency_gens = net.gen.index.tolist()

for timestamp in common_timestamps:
    # 4.1: Set base case loads/generation
    load_row = distributed_loads_df.loc[timestamp]
    gen_row = distributed_generation_df.loc[timestamp]
    
    # Assign loads
    for i in net.load.index:
        col_name = f"load_{i}"     
        if col_name in load_row:
            net.load.at[i, "p_mw"] = load_row[col_name]
        else:
            net.load.at[i, "p_mw"] = 0.0
    
    # Assign net.gen
    for i in net.gen.index:
        col_name = f"gen_{i}"    
        if col_name in gen_row:
            net.gen.at[i, "p_mw"] = gen_row[col_name]
        else:
            net.gen.at[i, "p_mw"] = 0.0
    
    # Assign net.sgen 
    for i in net.sgen.index:
        col_name = f"sgen_{i}"
        if col_name in gen_row:
            net.sgen.at[i, "p_mw"] = gen_row[col_name]
        else:
            net.sgen.at[i, "p_mw"] = 0.0

    # 4.2: Run power flow for base case
    try:
        pp.runpp(net)
        base_case_converged = True
#         print("Base case solved.")
    except:
        base_case_converged = False
#         print("Base case not solved.")
    
    # Initialize placeholders
    status = "secure"
    
    max_line_loading_base = np.nan
    min_bus_voltage_base = np.nan
    max_bus_voltage_base = np.nan
    
    max_line_loading_cont = np.nan
    min_bus_voltage_cont = np.nan
    max_bus_voltage_cont = np.nan
    
    if base_case_converged:
        # Check base-case voltage and line loadings
        max_line_loading_base = net.res_line.loc[net.res_line.index != 45, "loading_percent"].max()
        min_bus_voltage_base = net.res_bus.vm_pu.min()
        max_bus_voltage_base = net.res_bus.vm_pu.max()

        # Base-case security criteria
        if (max_line_loading_base > 100) or (min_bus_voltage_base < 0.9) or (max_bus_voltage_base > 1.1):
#             print("Base case constraint violation.")
            status = "insecure"
        else:
            # 4.3: If base case is secure, do N-1 checks
            worst_line_loading_n1 = 0.0
            worst_min_voltage_n1 = 1e6  # track global min
            worst_max_voltage_n1 = 0.0  # track global max

            # --- (A) Lines ---
            for line_idx in contingency_lines:
                net_copy = copy.deepcopy(net)
                # Disable the line in net_copy
                net_copy.line.at[line_idx, "in_service"] = False
                
                try:
                    pp.runpp(net_copy)
#                     print("Contingency line solved.")
                    # Evaluate results
                    max_line_loading_temp = net_copy.res_line.loc[net_copy.res_line.index != 45, "loading_percent"].max()
                    min_bus_voltage_temp = net_copy.res_bus.vm_pu.min()
                    max_bus_voltage_temp = net_copy.res_bus.vm_pu.max()
                    
                    # Track worst-case line loading
                    if max_line_loading_temp > worst_line_loading_n1:
                        worst_line_loading_n1 = max_line_loading_temp
                        
                    # For voltages, we store the "global" worst min and max
                    if min_bus_voltage_temp < worst_min_voltage_n1:
                        worst_min_voltage_n1 = min_bus_voltage_temp
                    if max_bus_voltage_temp > worst_max_voltage_n1:
                        worst_max_voltage_n1 = max_bus_voltage_temp
                    
                    # Check thresholds
                    if (max_line_loading_temp > 100) or (min_bus_voltage_temp < 0.9) or (max_bus_voltage_temp > 1.1):
#                         print(f"Max line overloading:{max_line_loading_temp}")
#                         print(f"Min bus voltage:{min_bus_voltage_temp}")
#                         print(f"Max bus voltage:{max_bus_voltage_temp}")
#                         print("Contingency line constraint violation.")
                        status = "insecure"
                        break
                except:
#                     print("Contingency line not solved.")
                    status = "insecure"
                    break
             
            # --- (B) Generators (gen) ---
            # Only proceed if still secure after lines
            if status == "secure":
                for gen_idx in contingency_gens:
                    net_copy = copy.deepcopy(net)
                    net_copy.gen.at[gen_idx, "in_service"] = False
                    try:
                        pp.runpp(net_copy)
#                         print("Contingency gen solved.")
                        max_line_loading_temp = net_copy.res_line.loc[net_copy.res_line.index != 45, "loading_percent"].max()
                        min_bus_voltage_temp = net_copy.res_bus.vm_pu.min()
                        max_bus_voltage_temp = net_copy.res_bus.vm_pu.max()
                        
                        if max_line_loading_temp > worst_line_loading_n1:
                            worst_line_loading_n1 = max_line_loading_temp
                            
                        if min_bus_voltage_temp < worst_min_voltage_n1:
                            worst_min_voltage_n1 = min_bus_voltage_temp
                        if max_bus_voltage_temp > worst_max_voltage_n1:
                            worst_max_voltage_n1 = max_bus_voltage_temp
                        
                        if (max_line_loading_temp > 100) or (min_bus_voltage_temp < 0.9) or (max_bus_voltage_temp > 1.1):
#                             print("Contingency gen constraint violation.")
                            status = "insecure"
                            break
                    except:
#                         print("Contingency gen not solved.")
                        status = "insecure"
                        break
            
            # If we survived all contingencies securely, record worst loading
            if status == "secure":
                max_line_loading_cont = worst_line_loading_n1
                min_bus_voltage_cont = worst_min_voltage_n1
                max_bus_voltage_cont = worst_max_voltage_n1

    else:
        # Base case didn't converge => insecure
        status = "insecure"

    # 4.4: Store results
    features["timestamp"].append(timestamp)
    features["status"].append(status)
    
    features["max_line_loading_percent_basecase"].append(max_line_loading_base)
    features["min_bus_voltage_pu_basecase"].append(min_bus_voltage_base)
    features["max_bus_voltage_pu_basecase"].append(max_bus_voltage_base)

    features["max_line_loading_percent_contingency"].append(max_line_loading_cont)
    features["min_bus_voltage_pu_contingency"].append(min_bus_voltage_cont)
    features["max_bus_voltage_pu_contingency"].append(max_bus_voltage_cont)

    for i in net.load.index:
        features[f"load_{i}_p_mw"].append(net.load.at[i, "p_mw"])

    for i in net.gen.index:
        features[f"gen_{i}_p_mw"].append(net.gen.at[i, "p_mw"])

    for i in net.sgen.index:
        features[f"sgen_{i}_p_mw"].append(net.sgen.at[i, "p_mw"])        

end_time = time.time()  # Record the end time

execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)
numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)
numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)
numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)
numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba 

In [ ]:
# ------------------------------
# 5. Create a DataFrame of Features
# ------------------------------
features_df = pd.DataFrame(features)
features_df.reset_index(drop=True, inplace=True)

# Save to CSV
features_df.to_csv("../../data/task1_security_classification/simulation_security_labels_n-1.csv", index=False)